In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.layers import * 
from common.gradient import numerical_gradient

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.parmas = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = {}
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        # x 입력 데이터, t 정답 레이블
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum( y== t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)    
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads 
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        return grads

In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
        
        if i % iter_per_epoch == 0:
            train_acc = network.accuracy(x_train, t_train)
            test_acc = network.accuracy(x_test, t_test)
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            print(train_acc, test_acc)

0.09928333333333333 0.1032
0.09928333333333333 0.1032
0.09915 0.1009
0.09915 0.1009
0.9270333333333334 0.9272
0.9269 0.9273
0.9244833333333333 0.9248
0.9242666666666667 0.9245
0.9490666666666666 0.9472
0.9491 0.9472
0.9492833333333334 0.9479
0.9493666666666667 0.9476
0.9577 0.9544
0.9576333333333333 0.9545
0.9569666666666666 0.9533
0.9568666666666666 0.9533
0.9662666666666667 0.9604
0.9663 0.9603
0.96625 0.9601
0.9662166666666666 0.9601
0.9714833333333334 0.9644
0.9715333333333334 0.9643
0.9715666666666667 0.9644
0.9716333333333333 0.9643
0.9743 0.9689
0.9742 0.9689
0.9739166666666667 0.9683
0.9738833333333333 0.9682
0.9759833333333333 0.9695
0.9759666666666666 0.9696
0.9754833333333334 0.9693
0.9754833333333334 0.9694
0.9780333333333333 0.9701
0.9780166666666666 0.97
0.97795 0.9694
0.97795 0.9695
0.9798833333333333 0.9702
0.9798333333333333 0.9701
0.9796 0.97
0.9796166666666667 0.9696
0.9831166666666666 0.972
0.9830666666666666 0.9719
0.9829833333333333 0.972
0.9829666666666667 0.9721